## Application

#### Duck/Rabbit classification

Goal : Apply transformations (flip and rotation) and evaluate improvement from vanilla quick mode training


## Setup Monk

#### Download Monk

In [0]:
!git clone https://github.com/Tessellate-Imaging/monk_v1

#### Setup dependencies

In [0]:
cd monk_v1

In [0]:
!pip install -r installation/requirements_cu10.txt

#### Import Monk

In [0]:
import os
import sys
sys.path.append("./monk_v1/monk");
import psutil
from pytorch_prototype import prototype

## Prepare Dataset

#### Download Dataset

In [0]:
!wget https://www.dropbox.com/s/6wxjjcy0ys95yhk/ducks_vs_rabbits.zip

#### Extract Dataset

In [0]:
!unzip ducks_vs_rabbits.zip

## Vanilla Training

In [0]:
!rm -rf workspace

#### Create experiment

In [0]:
ptf = prototype(verbose=1);
ptf.Prototype("ducks_vs_rabbits", "exp-1");

#### List Models to Choose from

In [0]:
#ptf.List_Models()

#### Load Dataset

In [0]:
ptf.Default(dataset_path="./ducks_vs_rabbits/", model_name="resnet18", freeze_base_network=True, num_epochs=5)

In [0]:
ptf.Summary()

#### Train

In [0]:
ptf.Train()

## Finetuning

#### Copy previous experiment

In [0]:
ptf.Prototype("ducks_vs_rabbits", "exp-2", copy_from=["ducks_vs_rabbits", "exp-1"]);

In [0]:
ptf.List_Activations()

In [0]:
ptf.append_dropout(probability=0.1);
ptf.append_softmax(final_layer=True);

#### Reset Transforms

In [0]:
ptf.reset_transforms();
ptf.reset_transforms(test=True);

#### List Transforms

In [0]:
ptf.List_Transforms()

#### Apply Updates

In [0]:
ptf.apply_grayscale(train=True, val=True, test=True);
ptf.apply_random_rotation(10, train=True, val=True);
ptf.apply_random_vertical_flip(probability=0.5, train=True, val=True);
ptf.apply_random_horizontal_flip(probability=0.8, train=True, val=True);
ptf.apply_random_perspective(distortion_scale=0.5, probability=0.5, train=True, val=True)

#### Reload after update

In [0]:
ptf.Reload()

#### Train

In [0]:
ptf.Train()

## Experiment Compare

#### Import Compare module

In [0]:
from compare_prototype import compare

#### Initialise Compare

In [0]:
ctf = compare(verbose=1);
ctf.Comparison("ducks_vs_rabbits_compare");

#### Add experiments

In [0]:
ctf.Add_Experiment("ducks_vs_rabbits", "exp-1");
ctf.Add_Experiment("ducks_vs_rabbits", "exp-2");

#### Generate Stats

In [0]:
ctf.Generate_Statistics();

In [0]:
!ls ./workspace/comparison/ducks_vs_rabbits_compare

#### Display Plots

In [0]:
from IPython.display import Image
#Image('./workspace/comparison/ducks_vs_rabbits_compare/stats_best_val_acc.png',width=500, height=500)
#Image('./workspace/comparison/ducks_vs_rabbits_compare/stats_max_gpu_usage.png',width=500, height=500)
#Image('./workspace/comparison/ducks_vs_rabbits_compare/stats_training_time.png',width=500, height=500)
#Image('./workspace/comparison/ducks_vs_rabbits_compare/train_accuracy.png',width=500, height=500)
Image('./workspace/comparison/ducks_vs_rabbits_compare/train_loss.png',width=700, height=500)
#Image('./workspace/comparison/ducks_vs_rabbits_compare/val_accuracy.png',width=500, height=500)
#Image('./workspace/comparison/ducks_vs_rabbits_compare/val_loss.png',width=500, height=500)

## Inference

#### Get the image

In [0]:
import requests
test_url = "https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/rabbit-breeds-american-white-1553635287.jpg?crop=0.976xw:0.651xh;0.0242xw,0.291xh&resize=480:*"

# URL of the image to be downloaded is defined as image_url 
r = requests.get(test_url) # create HTTP response object 
with open('rabbit.jpg','wb') as f: 
    f.write(r.content) 

In [0]:
import requests
test_url = "https://upload.wikimedia.org/wikipedia/commons/3/3f/Amerikanische_Pekingenten_2013_01%2C_cropped.jpg"

# URL of the image to be downloaded is defined as image_url 
r = requests.get(test_url) # create HTTP response object 
with open('duck.jpg','wb') as f: 
    f.write(r.content) 

#### Set experiment to Inference

In [0]:
ptf.Prototype("ducks_vs_rabbits", "exp-1", eval_infer=True);

#### Run predictions

In [0]:
img_name = "duck.jpg";
predictions = ptf.Infer(img_name=img_name, return_raw=True);
print(predictions)

In [0]:
img_name = "rabbit.jpg";
predictions = ptf.Infer(img_name=img_name, return_raw=True);
print(predictions)